FAKE NEWS DETECTOR | A SYSTEM TO IDENTIFY UNRELIABLE NEWS ARTICLES

In [118]:
import pandas as pd
import numpy as np

In [119]:
train=pd.read_csv('train.csv')

In [120]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [121]:
train=train.dropna()

In [122]:
x=train.drop('label',axis=1)

In [123]:
x.reset_index(inplace=True)

In [124]:
y=train['label']

In [125]:
voc_size=10000

In [126]:
x.shape

(18285, 5)

In [127]:
y.shape

(18285,)

DATA PREPROCESSING

In [128]:
import nltk
import re


In [129]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [130]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [131]:
ps=PorterStemmer()

In [132]:
corpus=[]

In [133]:
for i in range(0,len(x)):
  review=re.sub('[^a-zA-Z]',' ',x['title'][i])
  review=review.lower()
  review=review.split()
  
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [134]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

WORD EMBEDDING REPRESENTATION

In [135]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot

In [136]:
rep=[one_hot(i,voc_size) for i in corpus]

In [137]:
rep

[[2483, 1928, 1283, 9307, 4059, 9667, 1720, 5388, 7581, 782],
 [6028, 3891, 7027, 7411, 4442, 5125, 781],
 [2162, 5586, 4211, 6861],
 [6509, 6979, 3536, 5171, 6083, 3933],
 [7124, 4442, 6123, 9514, 5220, 3110, 4442, 638, 3770, 8522],
 [8291,
  9060,
  6541,
  9794,
  5300,
  7253,
  3557,
  2294,
  9268,
  8448,
  9835,
  896,
  7185,
  1707,
  781],
 [6498, 3547, 966, 554, 9022, 7999, 663, 5421, 1735, 4678, 7580],
 [7097, 4080, 5124, 2893, 5907, 8841, 7253, 7745, 1735, 4678, 7580],
 [6511, 2562, 6212, 4083, 9867, 4116, 262, 5388, 7253, 5548],
 [7769, 8336, 1364, 5477, 8148, 2802, 2620, 6131],
 [5751, 1662, 3668, 2876, 2493, 2565, 6126, 6547, 4760, 2853, 176],
 [5171, 4634, 4059, 4116, 7253, 5907],
 [8478, 874, 9196, 6976, 3107, 7697, 4207, 7822, 2809],
 [1443, 8104, 9106, 9298, 2645, 8013, 2023, 1735, 4678, 7580],
 [4243, 3617, 1519, 2810, 2903, 1735, 4678, 7580],
 [4425, 4133, 8985, 6421, 6001, 963, 1927, 1565, 3759, 2553],
 [9124, 3126, 3891],
 [8323, 5822, 7430, 5477, 7253, 845, 55

In [138]:
sent_length=50

In [139]:
embedded_docs=pad_sequences(rep,padding='pre',maxlen=sent_length)

In [140]:
embedded_docs

array([[   0,    0,    0, ..., 5388, 7581,  782],
       [   0,    0,    0, ..., 4442, 5125,  781],
       [   0,    0,    0, ..., 5586, 4211, 6861],
       ...,
       [   0,    0,    0, ..., 1735, 4678, 7580],
       [   0,    0,    0, ..., 1561, 3645, 4744],
       [   0,    0,    0, ..., 8761, 4238, 1170]], dtype=int32)

In [141]:
dim=50

LSTM | LONG SHORT TERM MEMORY MODEL

In [142]:
model=Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [143]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            500000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               120800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________


In [144]:
x_data=np.array(embedded_docs)
y_data=np.array(y)

In [145]:
x_data.shape


(18285, 50)

In [146]:
y_data.shape

(18285,)

SPLITTING OF DATA

In [147]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.30,random_state=42)

MODEL TRAINING

In [148]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
200/200 [==============================] - 7s 19ms/step - loss: 0.4777 - accuracy: 0.7435 - val_loss: 0.1865 - val_accuracy: 0.9172
Epoch 2/30
200/200 [==============================] - 3s 16ms/step - loss: 0.1222 - accuracy: 0.9515 - val_loss: 0.1923 - val_accuracy: 0.9216
Epoch 3/30
200/200 [==============================] - 3s 16ms/step - loss: 0.0788 - accuracy: 0.9722 - val_loss: 0.2227 - val_accuracy: 0.9216
Epoch 4/30
200/200 [==============================] - 3s 16ms/step - loss: 0.0554 - accuracy: 0.9804 - val_loss: 0.2480 - val_accuracy: 0.9200
Epoch 5/30
200/200 [==============================] - 3s 16ms/step - loss: 0.0370 - accuracy: 0.9884 - val_loss: 0.2813 - val_accuracy: 0.9054
Epoch 6/30
200/200 [==============================] - 3s 16ms/step - loss: 0.0328 - accuracy: 0.9891 - val_loss: 0.3287 - val_accuracy: 0.9141
Epoch 7/30
200/200 [==============================] - 3s 16ms/step - loss: 0.0201 - accuracy: 0.9942 - val_loss: 0.3449 - val_accuracy: 0.9149

PERFORMANCE METRICS AND ACCURACY

In [149]:
y_pred=model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [150]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [151]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [152]:
score=accuracy_score(y_test,y_pred)

In [153]:
score

0.9097703244622676

In [154]:
confusion_matrix(y_test,y_pred)

array([[2837,  270],
       [ 225, 2154]])

In [155]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.93      0.91      0.92      3107\n           1       0.89      0.91      0.90      2379\n\n    accuracy                           0.91      5486\n   macro avg       0.91      0.91      0.91      5486\nweighted avg       0.91      0.91      0.91      5486\n'